# First Test

In [ ]:
import infrared as ir

num_colors = 3
num_nodes = 4
edges = [(1,2),(1,4),(2,3),(3,4)]
cycles = [(1,2,3,4)]

## Set up the model

In [ ]:
model = ir.Model()

# dummy variable to avoid index transformations
model.add_variables(1,(0,0))

# add one variable per node
model.add_variables(num_nodes,num_colors)

# define NotEquals constraints in Python
ir.def_constraint_class('NotEquals',
    lambda i,j: [i,j], # signature and dependencies
    lambda x, y: x!=y)  # constraint semantics

# add one inequality constraint per edge 
model.add_constraints(NotEquals(i,j) for i,j in edges)

# define cardinality for sets of 4 variables
ir.def_function_class('Card', # define "cardinality"
    lambda i, j, k, l: [i,j,k,l], # signature, dependencies
    lambda x,y,z,w: len({x,y,z,w}))  # function evaluation

# add the cardinality function for every cycle
model.add_functions([Card(i,j,k,l) for i,j,k,l in cycles], 'card')

model.set_feature_weight(-1,'card') # set up in order to *minimize*

solver = ir.Optimizer(model)
print(f'Tree width: {solver.treewidth()}')

## Draw the dependency graph

In [ ]:
from IPython.display import Image
import re

# Plot dependency graph
filename = 'dependency_graph.dot'
model.write_graph(filename, True)

ir.dotfile_to_png(filename)
filename = re.sub(r"dot$","png",filename)

Image(filename=filename)

## Plot the tree decomposition

In [ ]:
filename="treedecomp"
solver.plot_td(filename,'png')
Image(filename=filename+".png")

## Generate colorings

In [ ]:
def assignment_to_coloring(a):
        colors=["red","green","blue"]
        coloring = {i:colors[v] for i,v in enumerate(a.values())}
        del coloring[0]
        return coloring

In [ ]:
def plot_coloring(coloring, filename):
    import graphviz

    G = graphviz.Graph('coloring', engine="fdp")

    for i in range(1,num_nodes+1):
        G.node(f'{i}', style="filled", color=coloring[i])
    for i,j in edges:
        G.edge(f'{i}',f'{j}')

    G.render(filename, format='png', cleanup=True)

## Sampling

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
 
solver = ir.Sampler(model)

### Existing sampling method

In [ ]:
for i in range(1, 26):
    print(f'\nSample #{i}')

    assignment = solver.sample_naive(non_redundant=False)
    coloring = assignment_to_coloring(assignment)

    plot_coloring(coloring, f'colored_graph')
    img = Image.open(f'colored_graph.png')
    plt.imshow(img)
    plt.show()

### Naive non-redundant sampling method

In [ ]:
for i in range(1, 26):
    print(f'\nSample #{i}')
    
    assignment = solver.sample_naive(non_redundant=True, repeats=1000)

    if assignment:
        coloring = assignment_to_coloring(assignment)

        plot_coloring(coloring, f'colored_graph')
        img = Image.open(f'colored_graph.png')
        plt.imshow(img)
        plt.show()